# Memotion

In this notebook, the Memotion dataset will be loaded and be separated in to test, train, and validation set. The Memotion dataset was recently released as part of the
SemEval-2020 shared task on ”Memotion Analysis”. The dataset contains 7000 memes, which is enough for our study.

## Setup

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import CLIPModel
import torch
import clip
from PIL import Image
import os

If the error "ImportError: cannot import name 'is_directory' from 'PIL._util'" occurs, please refer to the following link to downgrade your pillow:
https://www.datasciencelearner.com/importerror-cannot-import-name-pillow-version-from-pil-solved/

## Load the dataset, and have a look at the data

In [8]:
# load labels_pd_pickle
labels_df_pickle = pd.read_pickle('labels_pd_pickle')
# show first 5 rows
labels_df_pickle.head()

,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral


## Encode the images

Use CLIP to encode the images. CLIP Image Encoder is a transformer-based model that is optimized for image-text retrieval tasks. It is trained on a variety of image-text pairs, and is able to encode images into a vector representation. The vector representation can then be used to compute the similarity between images.

In [4]:
# Load the CLIP model and preprocessing function
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Load images and preprocess them for input to the model, store in a list
images = []
images_embeddings = []
# load all images in the directory
# set the path to the directory that contains the images
path = "./images"
# iterate through all the files in the directory
for filename in os.listdir(path):
    try:
        # check if the file is an image
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # open the image using PIL
            img = Image.open(os.path.join(path, filename))
            # preprocess the image
            img = preprocess(img).unsqueeze(0).to(device)
            # append the image to the list
            images.append(img)
    except IOError:
        print("Error: image file is truncated, skipping file:", filename)

Error: image file is truncated, skipping file: image_5119.png


c:\Users\86177\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
c:\Users\86177\anaconda3\lib\site-packages\PIL\Image.py:988: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [5]:
# Use the model to encode the image
#for image in images:
#    image_features = model.encode_image(image)
#    images_embeddings.append(image_features)
images_feature = model.encode_image(images[0])
print(images_feature)

tensor([[-4.4058e-01,  1.6021e-01, -8.1135e-01, -2.1535e-01, -4.1511e-01,
         -4.0899e-01, -1.8673e-01,  1.8744e-01,  1.8483e-01, -2.4182e-01,
          3.6646e-01,  2.7902e-01,  2.7146e-01, -1.0643e-01,  5.6995e-01,
          2.4080e-01,  2.3865e-01, -4.7276e-01,  8.7647e-02, -3.9217e-02,
          2.3974e-01, -9.7120e-02, -4.1330e-01, -2.7713e-01, -5.1899e-01,
          6.3505e-01, -4.0892e-01,  7.7369e-01,  3.6714e-01,  2.2544e-01,
         -3.1904e-01, -2.6136e-01, -1.4715e-01, -4.4940e-01, -2.8196e-01,
          2.8456e-01, -1.6911e-01, -4.2822e-01, -2.5313e-01, -8.4190e-01,
         -4.1998e-01,  3.3975e-02, -3.8952e-01,  2.4518e-01,  1.0283e-01,
          1.5419e+00,  4.1722e-02, -1.5198e-01,  6.3928e-01,  7.1754e-02,
         -5.8767e-01,  7.2152e-01,  4.0960e-01, -6.6864e-02, -7.4227e-01,
          1.4375e-01,  5.2660e-01,  2.2265e-01,  1.7411e-01, -1.9453e-01,
          1.8527e-02,  2.9663e-01, -1.4555e-01, -5.8480e-02,  3.3619e-01,
          2.4914e-02,  6.6902e-02,  6.